In [1]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from keras import backend as K
# os.environ["CUDA_VISIBLE_DEVICES"] = '1'
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
K.set_session(sess)
from keras import regularizers
from keras.layers import Input, Dense, Flatten, GlobalAveragePooling2D, Activation, Conv2D, MaxPooling2D, BatchNormalization, Lambda, Dropout
from keras.layers import SeparableConv2D, Add, Convolution2D, concatenate
from keras.models import Model, load_model, model_from_json
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam, SGD
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.utils import to_categorical
from sklearn import metrics
from sklearn.metrics import roc_curve, roc_auc_score, confusion_matrix, classification_report
import matplotlib.pyplot as plt
from scipy.optimize import brentq
from scipy.interpolate import interp1d
import glob
from PIL import Image
from tqdm import tqdm, trange
import random
from keras.applications import Xception, ResNet152
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

Using TensorFlow backend.


In [2]:
nb_classes = 2  # number of classes
img_width, img_height = 64, 64  # change based on the shape/structure of your images
batch_size = 128  # try 4, 8, 16, 32, 64, 128, 256 dependent on CPU/GPU memory capacity (powers of 2 values).
nb_epoch = 300  # number of iteration the algorithm gets trained.

In [3]:
train_dir = '/home/www/fake_detection/task2/128x128/train'
validation_dir = '/home/www/fake_detection/task2/128x128/validation'
test50_dir = '/home/www/fake_detection/task2/128x128/test_50'

In [4]:
model = ResNet152(include_top=True, weights=None, input_shape=(img_width, img_height, 3), classes=2)
model.summary()


Model: "resnet152"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 64, 64, 3)    0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 70, 70, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 32, 32, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 32, 32, 64)   256         conv1_conv[0][0]                 
_________________________________________________________________________________________

In [5]:
model.compile(optimizer=Adam(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

print(len(model.trainable_weights))

622


In [6]:
train_datagen = ImageDataGenerator(rescale=1./255,)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_dir,
                                                        target_size=(img_height, img_width),
                                                        batch_size=batch_size,
                                                        shuffle=True,
                                                        class_mode='categorical')

validation_generator = train_datagen.flow_from_directory(validation_dir,
                                                        target_size=(img_height, img_width),
                                                        batch_size=batch_size,
                                                        shuffle=False,
                                                        class_mode='categorical')

test50_generator = test_datagen.flow_from_directory(test50_dir,
                                                  target_size=(img_height, img_width),
                                                  batch_size=batch_size,
                                                  shuffle=False,
                                                  class_mode='categorical')

Found 128404 images belonging to 2 classes.
Found 32100 images belonging to 2 classes.
Found 37566 images belonging to 2 classes.


In [7]:
callback_list = [EarlyStopping(monitor='val_accuracy', patience=10),
                 ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3)]
history = model.fit_generator(train_generator,
                            steps_per_epoch=200,
                            epochs=100,
                            validation_data=validation_generator,
                            validation_steps=len(validation_generator),
                            callbacks=callback_list)


Epoch 1/100
200/200 [==============================] - 209s 1s/step - loss: 0.7272 - accuracy: 0.6608 - val_loss: 0.9450 - val_accuracy: 0.5153
Epoch 2/100
200/200 [==============================] - 188s 938ms/step - loss: 0.5092 - accuracy: 0.7855 - val_loss: 0.3806 - val_accuracy: 0.9748
Epoch 3/100
200/200 [==============================] - 199s 993ms/step - loss: 0.4409 - accuracy: 0.7988 - val_loss: 0.2962 - val_accuracy: 0.9913
Epoch 4/100
200/200 [==============================] - 197s 987ms/step - loss: 0.4095 - accuracy: 0.8087 - val_loss: 0.2628 - val_accuracy: 0.9938
Epoch 5/100
200/200 [==============================] - 195s 977ms/step - loss: 0.4023 - accuracy: 0.8128 - val_loss: 0.3861 - val_accuracy: 0.9945
Epoch 6/100
200/200 [==============================] - 196s 978ms/step - loss: 0.3968 - accuracy: 0.8109 - val_loss: 0.2552 - val_accuracy: 0.9900
Epoch 7/100
200/200 [==============================] - 198s 989ms/step - loss: 0.3931 - accuracy: 0.8093 - val_loss: 0.2

In [8]:
model.save('/home/www/fake_detection/model/celeba_resnetv2.h5')

In [9]:
model = load_model('/home/www/fake_detection/model/celeba_resnetv2.h5')

In [10]:
output = model.predict_generator(test50_generator, steps=len(test50_generator), verbose=1)
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
print(test50_generator.class_indices)
print(output)

294/294 [==============================] - 56s 191ms/step
{'0_real': 0, '1_fake': 1}
[[1.000 0.000]
 [1.000 0.000]
 [0.999 0.001]
 ...
 [0.065 0.935]
 [0.197 0.803]
 [0.194 0.806]]


In [11]:
output_score50 = []
output_class50 = []
answer_class50 = []
answer_class50_1 =[]

for i in trange(len(test50_generator)):
    output50 = model.predict_on_batch(test50_generator[i][0])
    output_score50.append(output50)
    answer_class50.append(test50_generator[i][1])
    
output_score50 = np.concatenate(output_score50)
answer_class50 = np.concatenate(answer_class50)

output_class50 = np.argmax(output_score50, axis=1)
answer_class50_1 = np.argmax(answer_class50, axis=1)

print(output_class50)
print(answer_class50_1)

100%|██████████| 294/294 [01:38<00:00,  2.99it/s]

[0 0 0 ... 1 1 1]
[0 0 0 ... 1 1 1]


In [12]:
cm50 = confusion_matrix(answer_class50_1, output_class50)
report50 = classification_report(answer_class50_1, output_class50)

recall50 = cm50[0][0] / (cm50[0][0] + cm50[0][1])
fallout50 = cm50[1][0] / (cm50[1][0] + cm50[1][1])

fpr50, tpr50, thresholds50 = roc_curve(answer_class50_1, output_score50[:, 1], pos_label=1.)
eer50 = brentq(lambda x : 1. - x - interp1d(fpr50, tpr50)(x), 0., 1.)
thresh50 = interp1d(fpr50, thresholds50)(eer50)

print(report50)
print(cm50)
print("AUROC: %f" %(roc_auc_score(answer_class50_1, output_score50[:, 1])))
print(thresh50)
print('test_acc: ', len(output_class50[np.equal(output_class50, answer_class50_1)]) / len(output_class50))

              precision    recall  f1-score   support

           0       0.91      0.78      0.84     18788
           1       0.81      0.93      0.86     18778

    accuracy                           0.85     37566
   macro avg       0.86      0.85      0.85     37566
weighted avg       0.86      0.85      0.85     37566

[[14680  4108]
 [ 1405 17373]]
AUROC: 0.913737
0.6057754158973125
test_acc:  0.8532449555449076
